In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import time
import pandas as pd

In [4]:

#loading embeddings and mappings
data = np.load(r'C:\Users\Darshan\Music\raman_kannan\NLP\word_embedding\data\word_embeddings.npy', mmap_mode='r')
with open(r'C:\Users\Darshan\Music\raman_kannan\NLP\word_embedding\data\mappings.json') as f:
    map_ = json.load(f)
data_set = zip(list(map_.values()),data)

#Convert to dict for mappings and word_emb to combine
dict_emb = {}
for k,v in data_set:
      dict_emb[k] = v

#Read csv dataframe
pydf = pd.read_csv(r'C:\Users\Darshan\Music\raman_kannan\NLP\word_embedding\data\CodeNetDataFrame_Sub.csv',index_col=0)
pydf['vectors'] = pydf['clean_docstring_tokens'].apply(lambda x: np.mean([dict_emb[word] for word in str(x).split() if word in dict_emb],axis=0))



c:\programdata\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [19]:
#Data Cleaning and Preparation for training 
df = pydf[pydf['vectors'].apply(lambda x: x.shape == (100,))]
array = [df['vectors'].iloc[i] for i in range(len(df))]
X = np.array(array)

In [7]:

num_clusters = 5
km = KMeans(n_clusters=num_clusters,random_state=42)
km.fit(X)
clusters = km.labels_.tolist()

print('Cluster Training Completed')


Cluster Training Completed


In [37]:
#NLP Part to input the human query with code snippet
from scipy.spatial.distance import cosine
import re

query = str(input('Enter human query\n'))

def tokenize(text):
    # obtains tokens with a least 1 alphabet
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return ' '.join(pattern.findall(text.lower()))

def convertToVector(x):
      return np.mean([dict_emb[word] for word in x.split() if word in dict_emb],axis=0)

def getResults(c,vec):
    sub_df = df[df['labels'] == c][['code','vectors','clean_docstring_tokens']]
    sub_df['cosine_score'] = sub_df['vectors'].apply(lambda x: 1-cosine(x,vec))
    sub_df.sort_values(by='cosine_score',ascending=False,inplace=True)
    return sub_df.iloc[0:5]

df['labels'] = clusters
q = tokenize(query)
query_vec = convertToVector(q)
query_cluster = km.predict([query_vec])
print(query_cluster[0])
recom = getResults(query_cluster[0],query_vec)
print('------------------------Code Recommendor----------------------------')
recom[['code','clean_docstring_tokens']]


Enter human query
os memory consumption solution


c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3
------------------------Code Recommendor----------------------------


,code,clean_docstring_tokens
24695,"def app_trim_memory(self, pid: int or str, lev...",trim memory
19575,"def free_memory(self):\r\n """"""Free memo...",free memory signal
3919,"def _print_memory(self, memory):\n """"""P...",print memory
4821,"def collect(self):\n """"""\n Colle...",collect memory stats
4586,"def collect(self):\n """"""\n Colle...",collect memory stats


In [ ]:
#pyspark code for rdd
#convert int to string
#key not found in dictionary
#how to build tree algorithms like random forests
#os memory consumption solution